# Code Function Playground

Use this notebook to invoke the key services directly (without the REST API) and inspect their outputs.
It clones the contents of `data/` into `data/_code_playground` so you can test safely.

In [1]:
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir
while not (project_root / "app").exists() and project_root != project_root.parent:
    project_root = project_root.parent

if not (project_root / "app").exists():
    raise RuntimeError("Could not locate project root containing 'app/'")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"🛠 Project root: {project_root}")

🛠 Project root: /Users/xiejiatong/Downloads/stock-fantasy-api


In [4]:
import json
import os
import shutil
from pathlib import Path

from dotenv import load_dotenv

from app.services.agent_service import AgentService
from app.services.data_service import DataService

load_dotenv(Path.cwd() / "app" / ".env")

BASE_DATA_DIR = Path("data")
PLAYGROUND_DIR = BASE_DATA_DIR / "_code_playground"
if PLAYGROUND_DIR.exists():
    shutil.rmtree(PLAYGROUND_DIR)
shutil.copytree(BASE_DATA_DIR, PLAYGROUND_DIR)
print(f"📁 Using sandboxed data dir: {PLAYGROUND_DIR}")

data_service = DataService(data_dir=str(PLAYGROUND_DIR))
agent_service = AgentService()
print("✅ Services ready")

📁 Using sandboxed data dir: data/_code_playground
✅ Services ready


## 1. Agent catalog output
Generate all 10 agents (5 styles × 2 models) and peek at their metadata.

In [5]:
agents = agent_service.generate_all_agents()
print(f"Total agents: {len(agents)}")
print(json.dumps(agents[:3], indent=2))

Total agents: 10
[
  {
    "id": 1,
    "model_name": "gpt-4o-mini",
    "model_provider": "ChatGPT",
    "style_name": "conservative",
    "style_description": "Capital preservation focus. Small positions, tight stops, low risk tolerance.",
    "system_prompt": "You are an autonomous trading agent managing a $10,000 portfolio of S&P 500 stocks.\n\nYour role is to:\n1. Analyze market data and technical indicators\n2. Make buy/sell/hold decisions for individual stocks\n3. Manage positions responsibly with strict risk controls\n4. Maximize risk-adjusted returns\n\nCRITICAL RULES:\n- You can only trade S&P 500 stocks\n- Maximum 5 concurrent positions\n- No single position > 30% of portfolio\n- No shorting - only long positions\n- All trades must include reasoning\n\nMARKET DATA PROVIDED:\n- Current stock prices and 3-month change %\n- Technical indicators: RSI, MACD, EMA\n- Account balance and current positions\n\nOUTPUT FORMAT (JSON):\n{\n  \"action\": \"buy\" | \"sell\" | \"hold\",\n  \

## 2. User + selection helpers
Create a sandbox user, select agents, and inspect the recorded events.

In [6]:
user = data_service.create_user("playground-user", "playground@example.com")
print("User record:\n", json.dumps(user, indent=2))

selection = data_service.record_agent_selection(
    user_id=user["user_id"],
    agent_ids=[agents[0]["id"], agents[2]["id"], agents[4]["id"]],
)
print("Selection event:\n", json.dumps(selection, indent=2))
print("Current roster:", data_service.get_user_agents(user["user_id"]))


SyntaxError: unterminated string literal (detected at line 2) (1042479554.py, line 2)

## 3. Trade logging + standings
Record a couple of trades and confirm the standings/user P&L helpers produce data.

In [ ]:
trade1 = data_service.record_trade({
    "agent_id": agents[0]["id"],
    "user_id": user["user_id"],
    "week_number": 1,
    "action": "buy",
    "stock_ticker": "PLAY",
    "quantity": 5,
    "price": 100.0,
    "pnl_delta": 0.0,
    "action_reason": "Opening position",
})

trade2 = data_service.record_trade({
    "agent_id": agents[0]["id"],
    "user_id": user["user_id"],
    "week_number": 1,
    "action": "sell",
    "stock_ticker": "PLAY",
    "quantity": 5,
    "price": 110.0,
    "pnl_delta": 50.0,
    "action_reason": "Take profits",
})

print("Logged trades:\n", json.dumps([trade1, trade2], indent=2))

standings = data_service.get_standings(agent_catalog=agents)
print("Standings sample:\n", json.dumps(standings[:3], indent=2))

pnl = data_service.get_user_pnl(user["user_id"])
print("User PnL:\n", json.dumps(pnl, indent=2))


## 4. Weekly snapshot helper
Persist the current standings snapshot to the sandboxed JSON file.

In [ ]:
snapshot = data_service.record_weekly_snapshot(week_number=1, standings=standings)
print(json.dumps(snapshot, indent=2))

## 5. Optional: live LLM probe
Run this only if your API keys are set; otherwise skip to avoid errors.

In [ ]:
import asyncio
from app.core.llm_service import LLMOrchestrator
from app.core.trading_styles import TRADING_STYLES

market_data = """
AAPL $210 (RSI 65)
MSFT $400 (RSI 55)
NVDA $125 (RSI 72)
""".strip()
account_state = {"cash": 10000, "positions": {}}

async def quick_probe():
    orchestrator = LLMOrchestrator()
    model_name = "deepseek-chat" if os.getenv("DEEPSEEK_API_KEY") else "gpt-4o-mini"
    decision = await orchestrator.get_decision(
        model_name=model_name,
        system_prompt=TRADING_STYLES["balanced"]["system_prompt"],
        market_data=market_data,
        account_state=json.dumps(account_state),
    )
    return decision.dict()

try:
    result = asyncio.run(quick_probe())
    print(json.dumps(result, indent=2))
except Exception as exc:
    print("LLM probe skipped:", exc)